Dataset com base https://www.kaggle.com/thedownhill/art-images-drawings-painting-sculpture-engraving/home

Context: 
Dataset for classifying different styles of art. Main categories have been taken here<br>
Content: 
5 kinds of data downloaded from google images, yandex images and this site:

Drawings and watercolours<br>
Works of painting<br>
Sculpture<br>
Graphic Art<br>
Iconography (old Russian art)<br>


Data is separated on training and validation sets

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os, shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [22]:
IMG_SIZE = 100

In [23]:
# filtro do erro para não exibir
import warnings
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

base_dir = './dataset'

train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'validation_set')

# criando os rotulos
labels = [x for x in os.listdir(train_dir)]

from PIL import Image

os.mkdir(os.path.join(base_dir, 'training_set_convert'))
os.mkdir(os.path.join(base_dir, 'validation_set_convert'))
train_dir_convert = os.path.join(base_dir, 'training_set_convert')
validation_dir_convert = os.path.join(base_dir, 'validation_set_convert')

for i in labels:
    os.mkdir(os.path.join(train_dir_convert, i))
    os.mkdir(os.path.join(validation_dir_convert, i))

for i in labels:
    _dir = os.path.join(train_dir, i)
    _dir_conv = os.path.join(train_dir_convert, i)
    for x in os.listdir(_dir):
        try:
            img = Image.open(os.path.join(_dir, x))
            img = img.resize((IMG_SIZE,IMG_SIZE), Image.ANTIALIAS)
            img.save(os.path.join(_dir_conv, x))
        except Exception:
            pass

for i in labels:
    _dir = os.path.join(validation_dir, i)
    _dir_conv = os.path.join(validation_dir_convert, i)
    for x in os.listdir(_dir):
        try:
            img = Image.open(os.path.join(_dir, x))
            img = img.resize((IMG_SIZE,IMG_SIZE), Image.ANTIALIAS)
            img.save(os.path.join(_dir_conv, x))
        except (Exception, UserWarning):
            pass


In [24]:
batch_size = 20

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = train_dir_convert
validation_dir = validation_dir_convert

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(IMG_SIZE,IMG_SIZE), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                              target_size=(IMG_SIZE,IMG_SIZE), 
                                                              batch_size=batch_size, 
                                                              class_mode='categorical')
            

Found 7718 images belonging to 5 classes.
Found 856 images belonging to 5 classes.


In [25]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               4333696   
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 645       
Total para

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=7818/batch_size, 
    epochs=10, 
    validation_data=validation_generator, 
    validation_steps=856/batch_size)

Epoch 1/10
310/390 [======================>.......] - ETA: 37s - loss: 0.9079 - acc: 0.6590

E:\Anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


391/390 [==============================] - 185s 473ms/step - loss: 0.8657 - acc: 0.6773 - val_loss: 0.6299 - val_acc: 0.7745
Epoch 2/10
391/390 [==============================] - 181s 462ms/step - loss: 0.6140 - acc: 0.7828 - val_loss: 0.6697 - val_acc: 0.7512
Epoch 3/10
391/390 [==============================] - 184s 469ms/step - loss: 0.5200 - acc: 0.8092 - val_loss: 0.5961 - val_acc: 0.7769
Epoch 4/10
391/390 [==============================] - 180s 461ms/step - loss: 0.4546 - acc: 0.8326 - val_loss: 0.5048 - val_acc: 0.8166
Epoch 5/10
391/390 [==============================] - 181s 463ms/step - loss: 0.4112 - acc: 0.8511 - val_loss: 0.5222 - val_acc: 0.8096
Epoch 6/10
391/390 [==============================] - 183s 467ms/step - loss: 0.3607 - acc: 0.8644 - val_loss: 0.4923 - val_acc: 0.8119
Epoch 7/10
391/390 [==============================] - 181s 463ms/step - loss: 0.3152 - acc: 0.8821 - val_loss: 0.5373 - val_acc: 0.8178
Epoch 8/10
391/390 [==============================] - 181s 

In [27]:
def tratar_img(img):
    img = cv2.imread(img)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    return img.reshape(1,IMG_SIZE,IMG_SIZE,3)

In [39]:
print(model.predict(tratar_img('desenho.jpg')))

print(labels)

[[1. 0. 0. 0. 0.]]
['drawings', 'engraving', 'iconography', 'painting', 'sculpture']
